In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from branchynet.net import BranchyNet
from branchynet.links import *
import chainer.functions as F
import chainer.links as L
from branchynet import utils, visualize
from chainer import cuda

Define Network

In [ ]:
from networks import alex_cifar10

branchyNet = alex_cifar10.get_network()
branchyNet.to_gpu()
branchyNet.training()

Import Data

In [ ]:
from datasets import mnist, pcifar10
x_train, y_train, x_test, y_test = pcifar10.get_data()

Settings

In [ ]:
TRAIN_BATCHSIZE = 512
TEST_BATCHSIZE = 128
TRAIN_NUM_EPOCHS = 10

Train Main Network

In [ ]:
main_loss, main_acc, main_time = utils.train(branchyNet, x_train, y_train, main=True, batchsize=TRAIN_BATCHSIZE,
                                             num_epoch=TRAIN_NUM_EPOCHS)

Train BranchyNet

In [ ]:
TRAIN_NUM_EPOCHS = 10
branch_loss, branch_acc, branch_time = utils.train(branchyNet, x_train, y_train, batchsize=TRAIN_BATCHSIZE,
                                                   num_epoch=TRAIN_NUM_EPOCHS)

#set network to inference mode
branchyNet.testing()

Visualizing Network Training

In [ ]:
visualize.plot_layers(main_loss, xlabel='Epochs', ylabel='Training Loss')
visualize.plot_layers(main_acc, xlabel='Epochs', ylabel='Training Accuracy')

In [ ]:
visualize.plot_layers(zip(*branch_loss), xlabel='Epochs', ylabel='Training Loss')
visualize.plot_layers(zip(*branch_acc), xlabel='Epochs', ylabel='Training Accuracy')

In [ ]:
import numpy as np
import dill
with open("_models/alexnet_cifar10_10epoch.bn", "wb") as f:
    dill.dump(branchyNet,f)

Get exit result, compute openmax vs softmax

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import dill
# with open("_models/alexnet_cifar10.bn", "rb") as f:
with open("_models/alexnet_cifar10_10epoch.bn", "rb") as f:
    branchyNet = dill.load(f)

In [3]:
from datasets import mnist, pcifar10
x_train, y_train, x_test, y_test = pcifar10.get_data()

In [4]:
# get all model output

In [5]:
x_train.shape

(50000, 3, 32, 32)

In [6]:
from branchynet.utils import model_test_batch
batch_size = 4000
train_feats = []
test_feats = []
branchyNet.to_gpu()
for i, model in enumerate(branchyNet.models):
    train_feat = model_test_batch(model,x_train,batch_size,gpu=True)
    test_feat = model_test_batch(model,x_test,batch_size,gpu=True)
    train_feats.append(train_feat)
    test_feats.append(test_feat)

In [7]:
train_feats = np.array(train_feats)
test_feats = np.array(test_feats)

In [8]:
from shadow.shadow import meta_model_labels,test_meta_model_labels
new_labels_train = meta_model_labels(y_train, train_feats)
new_labels_test = meta_model_labels(y_test, test_feats)

In [9]:
test_meta_model_labels(new_labels_train)
test_meta_model_labels(new_labels_test)

unqiue [0 1 2]
layer 0 drop		0 2526
layer 0 exit		1 39110
layer 0 continue	2 8364
layer 1 drop		0 2895
layer 1 exit		1 41195
layer 1 continue	2 5910
layer 2 drop		0 3882
layer 2 exit		1 46118
layer 2 continue	2 0
unqiue [0 1 2]
layer 0 drop		0 1351
layer 0 exit		1 7259
layer 0 continue	2 1390
layer 1 drop		0 1601
layer 1 exit		1 7576
layer 1 continue	2 823
layer 2 drop		0 2232
layer 2 exit		1 7768
layer 2 continue	2 0


In [13]:
from openmax.openmax import OpenMax, get_openmax_scores, plot_openmax_mean, expand_dims

openmax_fits_pass = []
openmax_fits_exit = []
openmax_fits_continue = []
x_train_openmaxs = []
x_test_openmaxs = []

for i,new_label in enumerate(new_labels_train):
    x_train_openmax_drop = train_feats[i][new_label==0]
    x_train_openmax_exit = train_feats[i][new_label==1]
    x_train_openmax_continue = train_feats[i][new_label==2]
    
    x_train_openmax_drop = expand_dims(x_train_openmax_drop)
    x_train_openmax_exit = expand_dims(x_train_openmax_exit)
    x_train_openmax_continue = expand_dims(x_train_openmax_continue)
    
    x_train_openmax = train_feats[i]
    x_train_openmax = expand_dims(x_train_openmax)    
    x_train_openmaxs.append(x_train_openmax)
    
    x_test_openmax = test_feats[i]
    x_test_openmax = expand_dims(x_test_openmax)    
    x_test_openmaxs.append(x_test_openmax)
    
#     if len(x_train_openmax_drop) > 0:
#         openmax = OpenMax(tailsize=10, distance_type="eucos", alpharank=3)
#         openmax.fit(x_train_openmax_drop)
#         openmax_fits_drop.append(openmax)
        
#         o_train, s_train = get_openmax_scores(x_train_openmax, openmax)
#         plot_openmax_mean(o_train, s_train, title='Exit {} Drop Train'.format(i))

#         o_test, s_test = get_openmax_scores(x_test_openmax, openmax)
#         plot_openmax_mean(o_test, s_test, title='Exit {} Drop Test'.format(i))

#     if len(x_train_openmax_exit) > 0:
#         openmax = OpenMax(tailsize=10, distance_type="eucos", alpharank=3)
#         openmax.fit(x_train_openmax_exit)
#         openmax_fits_exit.append(openmax)
    
#         o_train, s_train = get_openmax_scores(x_train_openmax, openmax)
#         plot_openmax_mean(o_train, s_train, title='Exit {} Exit Train'.format(i))
        
#         o_test, s_test = get_openmax_scores(x_test_openmax, openmax)
#         plot_openmax_mean(o_test, s_test, title='Exit {} Exit Test'.format(i))
        
#     if len(x_train_openmax_continue) > 0:
#         openmax = OpenMax(tailsize=10, distance_type="eucos", alpharank=3)
#         openmax.fit(x_train_openmax_continue)
#         openmax_fits_continue.append(openmax)
        
#         o_train, s_train = get_openmax_scores(x_train_openmax, openmax)
#         plot_openmax_mean(o_train, s_train, title='Exit {} Continue Train'.format(i))

#         o_test, s_test = get_openmax_scores(x_test_openmax, openmax)
#         plot_openmax_mean(o_test, s_test, title='Exit {} Continue Test'.format(i))
    
    

In [ ]:
from tqdm import tqdm
from scipy.stats import entropy

# Following a sample
k = 0 # exit k

x_data = x_test_openmaxs[k][:1000]
t_data = new_labels_test[k][:1000]

print "drop", np.sum(t_data==0)
print "exit", np.sum(t_data==1)
print "cont", np.sum(t_data==2)

count = 0.
total = len(x_data)
count_drop = 0
count_exit = 0
count_cont = 0
correct_drop_count = 0
correct_exit_count = 0
correct_cont_count = 0
ps = []
for x,t in tqdm(zip(x_data,t_data)):
    x = np.array([x])
    
    o_test_drop, s_test_drop = get_openmax_scores(x, openmax_fits_drop[k])
    o_test_exit, s_test_exit = get_openmax_scores(x, openmax_fits_exit[k])
    o_test_cont, s_test_cont = get_openmax_scores(x, openmax_fits_continue[k])

    # plot_openmax_mean(o_test_drop, s_test_drop, title='Exit {} Drop Test'.format(i))
    # plot_openmax_mean(o_test_exit, s_test_exit, title='Exit {} Exit Test'.format(i))
    # plot_openmax_mean(o_test_cont, s_test_cont, title='Exit {} Continue Test'.format(i))
    
    pl = np.argmin([o_test_drop[0][-1], o_test_exit[0][-1], o_test_cont[0][-1]])
    
    p = np.max(o_test_exit[0][0:-1])
#     p = entropy(o_test_exit[0])
    if np.argmax(o_test_exit)==10 and np.argmax(o_test_cont)==10:
        pl = 0
        count_drop=count_drop+1
#         pl = np.argmin([o_test_drop[0][-1], o_test_exit[0][-1], o_test_cont[0][-1]])
#         pl = np.argmin([entropy(o_test_drop[0]), entropy(o_test_exit[0]), entropy(o_test_cont[0])])

    elif np.argmax(o_test_exit)==10 and np.argmax(o_test_drop)==10:
        pl = 2
        count_cont=count_cont+1
    else:
        ps.append(p)
        pl=1
        count_exit=count_exit+1

    if t==pl:
        if pl==0:
            correct_drop_count=correct_drop_count+1
        if pl==1:
            correct_exit_count=correct_exit_count+1
        if pl==2:
            correct_cont_count=correct_cont_count+1
        count=count+1



In [ ]:
np.histogram(ps)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(ps)

print "correct_count",count

print "count_drop",count_drop
print "count_exit",count_exit
print "count_cont",count_cont

print "correct_drop_count",correct_drop_count
print "correct_exit_count",correct_exit_count
print "correct_cont_count",correct_cont_count

print "overall acc",100.0*count/total
print "drop acc",100.0*correct_drop_count/count_drop
print "exit acc",100.0*correct_exit_count/count_exit
print "cont acc",100.0*correct_cont_count/count_cont

In [14]:
from shadow.shadow import train_metamodel

# k=0

for k in range(1,3):
    meta_train = zip(x_train_openmaxs[k][:,0,:].astype(np.float32), np.array(new_labels_train[k],dtype=np.int32))
    meta_test = zip(x_test_openmaxs[k][:,0,:].astype(np.float32), np.array(new_labels_test[k],dtype=np.int32))
    meta_model0 = train_metamodel(meta_train, meta_test, 3, nodes=3000, epochs=10)

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy
1           1.26666     0.667722              0.800881       0.755469                  
     total [#####.............................................] 10.24%
this epoch [#.................................................]  2.40%
       200 iter, 1 epoch / 10 epochs
    3.2597 iters/sec. Estimated time to finish: 0:08:57.824494.
     total [#######...........................................] 15.36%
this epoch [##########################........................] 53.60%
       300 iter, 1 epoch / 10 epochs
    3.3753 iters/sec. Estimated time to finish: 0:08:09.768492.
2           0.436743    0.646236              0.82524        0.753516                  
     total [##########........................................] 20.48%
this epoch [##................................................]  4.80%
       400 iter, 2 epoch / 10 epochs
    3.3721 iters/sec. Estimated time to finish: 0:07:40.583567.
     tota